In [11]:
# === 1) Setup & load ===
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error

# FICHIER À ADAPTER SI BESOIN
DATA = Path("data/raw/velib_timeseries_5min.csv")

FREQ_MIN = 5
HORIZONS = [15, 30, 60]      # minutes
SPLIT_TRAINTEST = 0.70       # 70% / 30% (split temporel)
SPLIT_TRAINVAL  = 0.85       # 85% / 15% (dans TRAIN pour early stopping)

# Chargement
usecols = ["ts","station_id","bikes_available","capacity"]
dtypes  = {"station_id":"category","bikes_available":"float32","capacity":"float32"}
df = pd.read_csv(DATA, usecols=usecols, dtype=dtypes, parse_dates=["ts"])\
       .sort_values(["station_id","ts"]).reset_index(drop=True)

# Capacité robuste par station (pour ratio & clip)
df["capacity"] = (df.groupby("station_id")["capacity"]
                    .transform(lambda s: s.ffill().bfill().fillna(s.max()))
                    .astype("float32"))
df = df[df["capacity"] > 0].copy()

# Ratio d'occupation
df["occ"] = (df["bikes_available"] / df["capacity"]).clip(0, 1).astype("float32")

print("Rows:", len(df), "| Stations:", df["station_id"].nunique())
df.head(3)


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/1551392377.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["capacity"] = (df.groupby("station_id")["capacity"]


Rows: 3304414 | Stations: 1468


,ts,station_id,bikes_available,capacity,occ
0,2025-08-21 17:45:00+00:00,1002059045,23.0,27.0,0.851852
1,2025-08-21 17:50:00+00:00,1002059045,25.0,27.0,0.925926
2,2025-08-21 17:55:00+00:00,1002059045,23.0,27.0,0.851852


In [12]:
# === 2) Baseline Naïve (bikes) ===
tmp = df.copy()
for h in HORIZONS:
    sh = h // FREQ_MIN
    tmp[f"occ_{h}"] = tmp.groupby("station_id")["occ"].shift(-sh).astype("float32")

cut = tmp["ts"].quantile(SPLIT_TRAINTEST)
test_naive = tmp[tmp["ts"] > cut].copy()

mae_naive_bikes = {}
for h in HORIZONS:
    y = test_naive[f"occ_{h}"].dropna()
    p_occ = test_naive.loc[y.index, "occ"]
    cap   = test_naive.loc[y.index, "capacity"]
    mae_naive_bikes[h] = float(mean_absolute_error(y*cap, p_occ*cap))

print("MAE Naïve (bikes):", {h: round(v,3) for h,v in mae_naive_bikes.items()})


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/1036027159.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tmp[f"occ_{h}"] = tmp.groupby("station_id")["occ"].shift(-sh).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/1036027159.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tmp[f"occ_{h}"] = tmp.groupby("station_id")["occ"].shift(-sh).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/1036027159.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. 

MAE Naïve (bikes): {15: 0.75, 30: 1.156, 60: 1.739}


In [13]:
# === 3) Features & targets (with momentum) ===
feat = df.copy().sort_values(["station_id","ts"]).reset_index(drop=True)

# Calendrier au temps t
feat["hour"] = feat["ts"].dt.hour.astype("uint8")
feat["dow"]  = feat["ts"].dt.dayofweek.astype("uint8")
feat["is_weekend"] = (feat["dow"]>=5).astype("uint8")
feat["hour_sin"] = np.sin(2*np.pi*feat["hour"]/24).astype("float32")
feat["hour_cos"] = np.cos(2*np.pi*feat["hour"]/24).astype("float32")
# 2e harmonique (jour)
feat["hour_sin2"] = np.sin(4*np.pi*feat["hour"]/24).astype("float32")
feat["hour_cos2"] = np.cos(4*np.pi*feat["hour"]/24).astype("float32")

# Rolling 3h (t-1) — tendance plus lente
feat["occ_roll_180"] = (
    feat.groupby("station_id")["occ"].shift(1)
        .rolling(36).mean().reset_index(level=0, drop=True)
).astype("float32")


# Helper
def shift(col, k): 
    return feat.groupby("station_id")[col].shift(k)

# Lags d'occupation (5,10,15,30,60)
for k in [1,2,3,6,12]:
    feat[f"occ_lag_{k*FREQ_MIN}"] = shift("occ", k).astype("float32")

# Rolling moyens (t-1): 1h, 2h
feat["occ_roll_60"]  = shift("occ", 1).rolling(12).mean().reset_index(level=0, drop=True).astype("float32")
feat["occ_roll_120"] = shift("occ", 1).rolling(24).mean().reset_index(level=0, drop=True).astype("float32")

# Deltas rapides (5/15) + longs (30/60)
feat["occ_delta_5"]  = (feat["occ"] - shift("occ", 1)).astype("float32")
feat["occ_delta_15"] = (feat["occ"] - shift("occ", 3)).astype("float32")
feat["occ_delta_30"] = (feat["occ"] - shift("occ", 6)).astype("float32")
feat["occ_delta_60"] = (feat["occ"] - shift("occ", 12)).astype("float32")

# EMAs (t-1) et momentum
feat["occ_ema_fast"] = (shift("occ",1).groupby(feat["station_id"]).apply(
    lambda s: s.ewm(alpha=0.5, adjust=False).mean()
).reset_index(level=0, drop=True)).astype("float32")
feat["occ_ema_slow"] = (shift("occ",1).groupby(feat["station_id"]).apply(
    lambda s: s.ewm(alpha=0.1, adjust=False).mean()
).reset_index(level=0, drop=True)).astype("float32")
feat["occ_momentum"] = (feat["occ_ema_fast"] - feat["occ_ema_slow"]).astype("float32")

# Cibles: occupation future
for h in HORIZONS:
    sh = h // FREQ_MIN
    feat[f"occ_{h}"] = feat.groupby("station_id")["occ"].shift(-sh).astype("float32")

print("feat shape:", feat.shape)
feat.head(3)


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  feat.groupby("station_id")["occ"].shift(1)
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return feat.groupby("station_id")[col].shift(k)
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

feat shape: (3304414, 30)


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  feat[f"occ_{h}"] = feat.groupby("station_id")["occ"].shift(-sh).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  feat[f"occ_{h}"] = feat.groupby("station_id")["occ"].shift(-sh).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/2016304851.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of p

,ts,station_id,bikes_available,capacity,occ,hour,dow,is_weekend,hour_sin,hour_cos,...,occ_delta_5,occ_delta_15,occ_delta_30,occ_delta_60,occ_ema_fast,occ_ema_slow,occ_momentum,occ_15,occ_30,occ_60
0,2025-08-21 17:45:00+00:00,1002059045,23.0,27.0,0.851852,17,3,0,-0.965926,-0.258819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.814815,0.703704
1,2025-08-21 17:50:00+00:00,1002059045,25.0,27.0,0.925926,17,3,0,-0.965926,-0.258819,...,0.074074,NaN,NaN,NaN,0.851852,0.851852,0.00000,0.666667,0.777778,0.703704
2,2025-08-21 17:55:00+00:00,1002059045,23.0,27.0,0.851852,17,3,0,-0.965926,-0.258819,...,-0.074074,NaN,NaN,NaN,0.888889,0.859259,0.02963,0.740741,0.777778,0.703704


In [14]:
# === 4) Split + encodings + occ_now ===
# Split temporel Train/Test
cut = feat["ts"].quantile(SPLIT_TRAINTEST)
train = feat[feat["ts"] <= cut].copy()
test  = feat[feat["ts"] >  cut].copy()

# Encodages station (sur TRAIN)
sta_mean = (train.groupby("station_id")["occ"].mean().rename("sta_mean_occ")).reset_index()
sta_hdh  = (train.assign(hour=train["ts"].dt.hour.astype("uint8"),
                         dow=train["ts"].dt.dayofweek.astype("uint8"))
                 .groupby(["station_id","dow","hour"])["occ"].median()
                 .rename("sta_hdh_occ")).reset_index()

def add_station_encodings(frame):
    out = frame.merge(sta_mean, on="station_id", how="left")
    out = out.merge(sta_hdh, left_on=["station_id","dow","hour"],
                           right_on=["station_id","dow","hour"], how="left")
    out["sta_hdh_occ"] = out["sta_hdh_occ"].fillna(out["sta_mean_occ"])
    return out

train = add_station_encodings(train).sort_values(["station_id","ts"]).reset_index(drop=True)
test  = add_station_encodings(test ).sort_values(["station_id","ts"]).reset_index(drop=True)

# occ_now (ancre)
train["occ_now"] = train["occ"].astype("float32")
test["occ_now"]  = test["occ"].astype("float32")

print("train/test:", train.shape, test.shape)
train.head(2)


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/3593776824.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sta_mean = (train.groupby("station_id")["occ"].mean().rename("sta_mean_occ")).reset_index()
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/3593776824.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["station_id","dow","hour"])["occ"].median()


train/test: (2313255, 33) (991159, 33)


,ts,station_id,bikes_available,capacity,occ,hour,dow,is_weekend,hour_sin,hour_cos,...,occ_delta_60,occ_ema_fast,occ_ema_slow,occ_momentum,occ_15,occ_30,occ_60,sta_mean_occ,sta_hdh_occ,occ_now
0,2025-08-21 17:45:00+00:00,1002059045,23.0,27.0,0.851852,17,3,0,-0.965926,-0.258819,...,NaN,NaN,NaN,NaN,0.666667,0.814815,0.703704,0.574086,0.740741,0.851852
1,2025-08-21 17:50:00+00:00,1002059045,25.0,27.0,0.925926,17,3,0,-0.965926,-0.258819,...,NaN,0.851852,0.851852,0.0,0.666667,0.777778,0.703704,0.574086,0.740741,0.925926


In [15]:
# === 5) Δ targets per split + dataset Δ ===
# Cibles Δ définies séparément dans TRAIN/TEST (pas de fuite)
for h in HORIZONS:
    sh = h // FREQ_MIN
    train[f"occ_delta_target_{h}"] = (train.groupby("station_id")["occ"].shift(-sh) - train["occ"]).astype("float32")
    test[f"occ_delta_target_{h}"]  = (test .groupby("station_id")["occ"].shift(-sh) - test ["occ"]).astype("float32")

# Liste de features Δ (sans encodages station au départ)
base_feats = [
    "dow","is_weekend","hour_sin","hour_cos","hour_sin2","hour_cos2",
    "occ_now",
    "occ_lag_5","occ_lag_10","occ_lag_15","occ_lag_30","occ_lag_60",
    "occ_roll_60","occ_roll_120","occ_roll_180",
    "occ_delta_5","occ_delta_15","occ_delta_30","occ_delta_60",
    # si tu as gardé les EMA : "occ_ema_fast","occ_ema_slow","occ_momentum",
]
feat_cols_delta = base_feats + ["capacity"]


# Nettoyage
need_train = feat_cols_delta + [f"occ_delta_target_{h}" for h in HORIZONS] + [f"occ_{h}" for h in HORIZONS]
need_test  = feat_cols_delta + [f"occ_delta_target_{h}" for h in HORIZONS] + [f"occ_{h}" for h in HORIZONS]

train_delta = train.dropna(subset=need_train).copy()
test_delta  = test .dropna(subset=need_test ).copy()

print("train_delta/test_delta:", train_delta.shape, test_delta.shape)


/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/3277451984.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train[f"occ_delta_target_{h}"] = (train.groupby("station_id")["occ"].shift(-sh) - train["occ"]).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/3277451984.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test[f"occ_delta_target_{h}"]  = (test .groupby("station_id")["occ"].shift(-sh) - test ["occ"]).astype("float32")
/var/folders/6r/y7n5k5ss483dvgr1svv3fq_80000gn/T/ipykernel_61798/3277451984.py:5: FutureWarning: The default of observed=False is de

train_delta/test_delta: (2242791, 36) (973555, 36)


In [16]:
# === Δ LightGBM avec calibration gamma sur la validation ===
# !pip install lightgbm -q
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error

def train_lgbm_delta_with_gamma(train_df, test_df, horizon, num_threads=2, gamma_grid=(0.5, 0.7, 0.9, 1.0)):
    y_col = f"occ_delta_target_{horizon}"
    tr = train_df.sort_values(["station_id","ts"]).reset_index(drop=True)
    te = test_df .sort_values(["station_id","ts"]).reset_index(drop=True)

    # Split temporel train/val
    cut_tr = tr["ts"].quantile(0.85)
    tr_tr  = tr[tr["ts"] <= cut_tr].copy()
    tr_val = tr[tr["ts"] >  cut_tr].copy()

    Xtr, ytr = tr_tr[feat_cols_delta], tr_tr[y_col]
    Xval, yval = tr_val[feat_cols_delta], tr_val[y_col]
    Xte        = te[feat_cols_delta]

    params = dict(
        objective="regression_l1",
        metric="l1",
        learning_rate=0.08,
        num_leaves=63,
        max_depth=-1,
        min_data_in_leaf=64,
        feature_fraction=0.85,
        bagging_fraction=0.8,
        bagging_freq=1,
        lambda_l1=0.0, lambda_l2=0.0,
        seed=42, verbosity=-1, num_threads=num_threads
    )

    dtrain = lgb.Dataset(Xtr, label=ytr)
    dval   = lgb.Dataset(Xval, label=yval, reference=dtrain)

    model = lgb.train(
        params, dtrain, num_boost_round=800,
        valid_sets=[dtrain, dval], valid_names=["train","val"],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    # --- Calibration gamma sur la validation (MAE en vélos) ---
    delta_val = model.predict(Xval, num_iteration=model.best_iteration)
    cap_val   = Xval["capacity"].to_numpy()
    occ_now_v = tr_val["occ_now"].to_numpy()
    y_true_v  = (tr_val[f"occ_{horizon}"] * cap_val).to_numpy()

    best_g, best_mae = None, 1e9
    for g in gamma_grid:
        occ_hat_v = np.clip(occ_now_v + g*delta_val, 0, 1)
        y_hat_v   = occ_hat_v * cap_val
        mae_v     = mean_absolute_error(y_true_v, y_hat_v)
        if mae_v < best_mae:
            best_mae, best_g = mae_v, g

    # --- Prédiction test avec gamma* ---
    delta_te = model.predict(Xte, num_iteration=model.best_iteration)
    cap_te   = Xte["capacity"].to_numpy()
    occ_now_t= te["occ_now"].to_numpy()
    y_true_t = (te[f"occ_{horizon}"] * cap_te).to_numpy()
    occ_hat_t= np.clip(occ_now_t + best_g*delta_te, 0, 1)
    y_hat_t  = occ_hat_t * cap_te
    mae_t    = mean_absolute_error(y_true_t, y_hat_t)

    return mae_t, model, int(model.best_iteration or 0), float(best_g)


In [17]:
mae_lgbm_delta, iters_delta, models_delta, gammas = {}, {}, {}, {}
for h in HORIZONS:
    mae, mdl, it, g = train_lgbm_delta_with_gamma(train_delta, test_delta, h, num_threads=2)
    mae_lgbm_delta[h] = mae; iters_delta[h] = it; models_delta[h] = mdl; gammas[h] = g

print("Best iters (Δ, gamma-tuned):", iters_delta)
print("Gammas:", gammas)
print("MAE LGBM (Δ->bikes, gamma):", {h: round(v,3) for h,v in mae_lgbm_delta.items()})

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[477]	train's l1: 0.0261078	val's l1: 0.034214
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[288]	train's l1: 0.0400235	val's l1: 0.050609
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[793]	train's l1: 0.0586823	val's l1: 0.0734025
Best iters (Δ, gamma-tuned): {15: 477, 30: 288, 60: 793}
Gammas: {15: 1.0, 30: 1.0, 60: 1.0}
MAE LGBM (Δ->bikes, gamma): {15: 0.754, 30: 1.156, 60: 1.714}


In [18]:
h = 30
te_dbg = test_delta.dropna(subset=feat_cols_delta+[f"occ_{h}", f"occ_delta_target_{h}"])\
                   .sort_values(["station_id","ts"]).reset_index(drop=True)
mdl = models_delta[h]
delta_pred = mdl.predict(te_dbg[feat_cols_delta], num_iteration=mdl.best_iteration)
g = gammas[h]
occ_hat = np.clip(te_dbg["occ_now"].to_numpy() + g*delta_pred, 0, 1)
y_hat = occ_hat * te_dbg["capacity"].to_numpy()
y_true = te_dbg[f"occ_{h}"].to_numpy() * te_dbg["capacity"].to_numpy()

for i in range(3):
    row = te_dbg.iloc[i][["ts","station_id","bikes_available","capacity"]].to_dict()
    print(row, " | y_true:", round(float(y_true[i]),2), " | y_hat:", round(float(y_hat[i]),2), " | gamma:", g)


{'ts': Timestamp('2025-08-28 23:15:00+0000', tz='UTC'), 'station_id': '1002059045', 'bikes_available': 5.0, 'capacity': 27.0}  | y_true: 4.0  | y_hat: 5.0  | gamma: 1.0
{'ts': Timestamp('2025-08-28 23:20:00+0000', tz='UTC'), 'station_id': '1002059045', 'bikes_available': 4.0, 'capacity': 27.0}  | y_true: 5.0  | y_hat: 4.0  | gamma: 1.0
{'ts': Timestamp('2025-08-28 23:25:00+0000', tz='UTC'), 'station_id': '1002059045', 'bikes_available': 5.0, 'capacity': 27.0}  | y_true: 4.0  | y_hat: 5.0  | gamma: 1.0


In [19]:
import json, pandas as pd, numpy as np
from pathlib import Path

ART = Path("artifacts/v0_1")
ART.mkdir(parents=True, exist_ok=True)

# 1) Sauve les boosters LightGBM (un par horizon)
import lightgbm as lgb
for h, mdl in models_delta.items():
    mdl.save_model(str((ART / f"lgbm_delta_h{h}.txt").resolve()))

# 2) Liste des features (dans le même ordre qu'à l'entraînement)
with open(ART / "feat_cols_delta.json", "w") as f:
    json.dump(feat_cols_delta, f)

# 3) Config d’entraînement
config = {
    "version": "v0_1",
    "freq_min": FREQ_MIN,
    "horizons": HORIZONS,
    "split_train_test": float(SPLIT_TRAINTEST),
    "split_train_val": float(SPLIT_TRAINVAL),
    "gammas": gammas,           # {15:1.0, 30:1.0, 60:1.0} dans ton cas
}
with open(ART / "config.json", "w") as f:
    json.dump(config, f, indent=2)

# 4) Métriques finales
pd.DataFrame({
    "horizon_min": HORIZONS,
    "mae_naive":   [mae_naive_bikes[h]  for h in HORIZONS],
    "mae_model":   [mae_lgbm_delta[h]   for h in HORIZONS],
    "best_iter":   [iters_delta[h]      for h in HORIZONS],
    "gamma":       [gammas[h]           for h in HORIZONS],
}).to_csv(ART / "metrics.csv", index=False)

print("Artifacts saved to:", ART.resolve())


Artifacts saved to: /Users/sako/Documents/PROJET DISPO VELIB/artifacts/v0_1


In [20]:
import json, pandas as pd, numpy as np, lightgbm as lgb
from pathlib import Path

ART = Path("artifacts/v0_1")
feat_cols_delta = json.load(open(ART / "feat_cols_delta.json"))
gammas = json.load(open(ART / "config.json"))["gammas"]

def load_model(h: int):
    return lgb.Booster(model_file=str(ART / f"lgbm_delta_h{h}.txt"))

def predict_batch(df_features: pd.DataFrame, h: int) -> np.ndarray:
    booster = load_model(h)
    X = df_features[feat_cols_delta]
    delta = booster.predict(X, num_iteration=booster.best_iteration)
    occ_hat = np.clip(df_features["occ_now"].to_numpy() + gammas[str(h)]*delta, 0, 1)
    return occ_hat * df_features["capacity"].to_numpy()
